In [24]:
import jax
import jax.numpy as np
import numpy as onp
from jax import random, jit, vmap, grad
from jax.experimental import optimizers,stax
from jax.experimental.stax import Dense, Conv, Relu, MaxPool, Flatten, Softmax,BatchNorm,Dropout,AvgPool
from jax.experimental.optimizers import optimizer, make_schedule, exponential_decay
#from jax.ops import index, index_update

import itertools
from functools import partial
from tqdm import trange
from torch.utils import data
import matplotlib.pyplot as plt

from scipy.integrate import odeint

from jax.nn import sigmoid,relu,log_sigmoid,one_hot
from jax.lax import scan
import h5py
from sklearn.metrics import confusion_matrix

In [25]:
import functools
import operator as op

from jax import lax
from jax import random
import jax.numpy as jnp

from jax.nn import (relu, log_softmax, softmax, softplus, sigmoid, elu,
                    leaky_relu, selu, gelu)
from jax.nn.initializers import glorot_normal, normal, ones, zeros

In [26]:
def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm/ np.sum(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2%' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [27]:
from keras.datasets import cifar10
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [28]:
class DataGenerator(data.Dataset):
    def __init__(self, images, labels, 
                 batch_size=128, 
                 rng_key=random.PRNGKey(1234)):
        'Initialization'
        self.images = images
        self.labels = labels
        self.N = labels.shape[0]
        self.batch_size = batch_size
        self.key = rng_key

    @partial(jit, static_argnums=(0,))
    def __data_generation(self, key, images, labels):
        'Generates data containing batch_size samples'
        idx = random.choice(key, self.N, (self.batch_size,), replace=False)
        images = images[idx,...]
        labels = labels[idx,...]
        return images, labels

    def __getitem__(self, index):
        'Generate one batch of data'
        self.key, subkey = random.split(self.key)
        images, labels = self.__data_generation(self.key, self.images, self.labels)
        return images, labels

In [29]:
x_train=np.asarray(x_train)
x_test=np.asarray(x_test)
x_train=x_train/255.
x_test=x_test/255.

In [30]:
#RGBtogray=np.array([0.299,0.587,0.114])
#x_train_gray = np.dot(x_train[:,:,:,:3], RGBtogray)
#x_test_gray = np.dot(x_test[:,:,:,:3], RGBtogray)


#x_train_gray = x_train_gray.reshape(-1,32,32,1)
#x_test_gray = x_test_gray.reshape(-1,32,32,1)


In [31]:
num_classes=y_train.max()+1
y_train=y_train.flatten()
y_train=one_hot(y_train,num_classes)

print(x_train.shape)
print(y_train.shape)

train_dataset = DataGenerator(x_train, y_train, batch_size=256)

(50000, 32, 32, 3)
(50000, 10)


In [32]:
num_classes=y_test.max()+1
y_test=y_test.flatten()
y_test=one_hot(y_test,num_classes)
test_dataset = DataGenerator(x_test, y_test, batch_size=256)

In [33]:
print(x_test.shape)
print(y_test.shape)

(10000, 32, 32, 3)
(10000, 10)


In [34]:
# Architecture
def VGG16():
    init, apply = stax.serial(Conv(64, (3, 3),padding="SAME"), #1
                              Relu, 
                              Conv(64, (3, 3),padding="SAME"), #2
                              Relu,
                              MaxPool((2, 2), (2, 2)),
                              Conv(128, (3,3),padding="SAME"), #3
                              Relu,
                              Conv(128, (3,3),padding="SAME"), #4
                              Relu,
                              MaxPool((2, 2), (2, 2)),
                              Conv(256, (3,3),padding="SAME"), #5
                              Relu,
                              Conv(256, (3,3),padding="SAME"), #6
                              Relu,
                              Conv(256, (3,3),padding="SAME"), #7
                              Relu,
                              MaxPool((2, 2), (2, 2)),
                              Conv(512, (3,3),padding="SAME"), #8
                              Relu,
                              Conv(512, (3,3),padding="SAME"), #9
                              Relu,
                              Conv(512, (3,3),padding="SAME"), #10
                              Relu,
                              MaxPool((2, 2), (2, 2)),
                              Conv(512, (3,3),padding="SAME"), #11
                              Relu,
                              Conv(512, (3,3),padding="SAME"), #12
                              Relu,
                              Conv(512, (3,3),padding="SAME"), #13
                              Relu,
                              MaxPool((2, 2), (2, 2)),
                              Flatten,
                              Dense(512), 
                              Relu, 
                              Dense(10), 
                              Softmax)
    return init, apply

In [35]:
class CNNclassifier:
    # Initialize the class
    def __init__(self, rng_key=random.PRNGKey(0)):
        # MLP init and apply functions
        self.net_init, self.net_apply = VGG16()
        _, params = self.net_init(rng_key, (256,32,32,3))

        # Optimizer initialization and update functions
        lr = optimizers.exponential_decay(1e-3, decay_steps=1000, decay_rate=0.999)
        self.opt_init, \
        self.opt_update, \
        self.get_params = optimizers.adam(lr)
        self.opt_state = self.opt_init(params)

        # Logger
        self.itercount = itertools.count()
        self.loss_log = []

    def loss(self, params, batch,it):
        images, labels = batch
        outputs = self.net_apply(params, images,rng=random.PRNGKey(it))
        loss = -labels*np.log(outputs+1e-7)
        return np.mean(loss)

    @partial(jit, static_argnums=(0,))
    def step(self, i, opt_state, batch):
        params = self.get_params(opt_state)
        gradients = grad(self.loss)(params, batch, i)
        return self.opt_update(i, gradients, opt_state)

    def train(self, dataset, nIter = 10):
        data = iter(dataset)
        pbar = trange(nIter)
        # Main training loop
        for it in pbar:
            # Run one gradient descent update
            batch = next(data)
            self.opt_state = self.step(next(self.itercount), self.opt_state, batch)  
            if it % 50 == 0:
                # Logger
                params = self.get_params(self.opt_state)
                loss = self.loss(params, batch,it)
                self.loss_log.append(loss)
                pbar.set_postfix({'Loss': loss})

    @partial(jit, static_argnums=(0,))
    def predict(self, params, inputs):
        outputs = self.net_apply(params, inputs,rng=random.PRNGKey(0))
        return outputs

In [36]:
# Initialize model
model = CNNclassifier()

In [37]:
# Architecture
def VGG16_test():
    init, apply = stax.serial(Conv(64, (3, 3),padding="SAME"), #1
                              Relu, 
                              BatchNorm(),
                              Dropout(0.7,mode='test'),
                              Conv(64, (3, 3),padding="SAME"), #2
                              Relu,
                              BatchNorm(),
                              MaxPool((2, 2), (2, 2)),
                              Conv(128, (3,3),padding="SAME"), #3
                              Relu,
                              BatchNorm(),
                              Dropout(0.6,mode='test'),
                              Conv(128, (3,3),padding="SAME"), #4
                              Relu,
                              BatchNorm(),
                              MaxPool((2, 2), (2, 2)),
                              Conv(256, (3,3),padding="SAME"), #5
                              Relu,
                              BatchNorm(),
                              Dropout(0.6,mode='test'),
                              Conv(256, (3,3),padding="SAME"), #6
                              Relu,
                              BatchNorm(),
                              Dropout(0.6,mode='test'),
                              Conv(256, (3,3),padding="SAME"), #7
                              Relu,
                              BatchNorm(),
                              MaxPool((2, 2), (2, 2)),
                              Conv(512, (3,3),padding="SAME"), #8
                              Relu,
                              BatchNorm(),
                              Dropout(0.6,mode='test'),
                              Conv(512, (3,3),padding="SAME"), #9
                              Relu,
                              BatchNorm(),
                              Dropout(0.6,mode='test'),
                              Conv(512, (3,3),padding="SAME"), #10
                              Relu,
                              BatchNorm(),
                              MaxPool((2, 2), (2, 2)),
                              Conv(512, (3,3),padding="SAME"), #11
                              Relu,
                              BatchNorm(),
                              Dropout(0.6,mode='test'),
                              Conv(512, (3,3),padding="SAME"), #12
                              Relu,
                              BatchNorm(),
                              Dropout(0.6,mode='test'),
                              Conv(512, (3,3),padding="SAME"), #13
                              Relu,
                              BatchNorm(),
                              MaxPool((2, 2), (2, 2)),
                              Dropout(0.5,mode='test'),
                              Flatten,
                              Dense(512), 
                              Relu, 
                              BatchNorm(axis=(0,1)),
                              Dropout(0.5,mode='test'),
                              Dense(10), 
                              Softmax)
    return init, apply

In [38]:
test_init,test_apply=VGG16()

In [39]:
# Train model
model.train(train_dataset, nIter=5000)
opt_params = model.get_params(model.opt_state)
# Plot loss
plt.figure()
plt.plot(model.loss_log, lw=2)
plt.yscale('log')
plt.xlabel('Iter #')
plt.ylabel('Loss')

 19%|█▉        | 952/5000 [1:00:00<4:27:40,  3.97s/it, Loss=0.23022561]

In [40]:
def predict(params, inputs):
  outputs = test_apply(params, inputs,rng=random.PRNGKey(0))
  return outputs

In [41]:
# Compute classification accuracy on the entire test data-set
@jit
def accuracy(params, batch):
    images, labels = batch
    outputs = predict(params, images)
    pred_class = np.argmax(outputs,1)
    true_class = np.argmax(labels,1)
    return np.sum((pred_class == true_class)) / images.shape[0]

acc = accuracy(opt_params, (x_test, y_test))
print('Classification accuracy: {}%'.format(100*acc))

Classification accuracy: 9.999999046325684%
